In [1]:
import clingo
import re

In [2]:
def copy_file_content(original_file, new_file):
    with open(original_file, 'r') as src:
        content = src.read()
    with open(new_file, 'w') as dst:
        dst.write(content)

In [3]:
# def filter(output_file):
#     with open(output_file, 'r') as output:
#         lines = output.readlines()
#         processed_lines = []

#         for line in lines:
#             line = line.strip()

#             # Remove incomplete lines that start with specific keywords and do not end with ")"
#             if (line.startswith(('neg', 'pos', 'relation')) and not line.endswith(')')):
#                 continue

#             # Add a full stop to lines that do not start with specific keywords
#             if not line.startswith(('neg', 'pos', 'relation')) and not line.endswith('.'):
#                 line += '.'

#             # Connect all space-separated words with underscores using regex
#             line = re.sub(r'(\w+) (\w+)', r'\1_\2', line)

#             processed_lines.append(line)

#     with open(output_file, 'w') as output:
#         output.write('\n'.join(processed_lines))

In [4]:
# def num_to_words(match):
#     num_word_dict = {'0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', 
#                      '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine', '10': 'ten', 
#                      '15': 'fifteen', '10': 'ten'}
#     return num_word_dict.get(match.group(), match.group())

# def filter(output_file):
#     with open(output_file, 'r') as output:
#         lines = output.readlines()
#     processed_lines = []

#     for line in lines:
#         line = line.strip()

#         # Skip lines that start with "section", ".", "*", or "`"
#         if line.startswith(("section", ".", "*", "`", "%")):
#             continue
#         # Remove special characters
#         line = re.sub(r'[\$\é]', '', line)
#         # Convert numbers to words
#         line = re.sub(r'\b(?:0|1|2|3|4|5|6|7|8|9|10|15)\b', num_to_words, line)
#         # Convert uppercase to lowercase
#         line = line.lower()
#         # Remove dots between words
#         line = re.sub(r'(?<=\w)\.(?=\w)', '', line)
        
#         # # Remove incomplete sentences
#         # if not line.endswith(')'):
#         #     continue
        
#         # Add "." where necessary
#         if line.startswith(('neg', 'pos', 'relation')) and not line.endswith('.'):
#             line += '.'
#         elif not line.startswith(('neg', 'pos', 'relation')) and not line.endswith('.'):
#             line += '.'

#         processed_lines.append(line)

#     with open(output_file, 'w') as output:
#         output.write('\n'.join(processed_lines))


In [28]:
def convert_number_to_words(s):
    num_words = {
        '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
        '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
    }
    # Replace each digit individually
    return ''.join(num_words.get(char, char) for char in s)

def replace_spaces_with_underscores_in_identifiers(s):
    # Precompile regular expression patterns for performance
    pattern_identifier = re.compile(r'\b([a-z_][a-z0-9_]*)\s+([a-z0-9_]+)\b', re.IGNORECASE)
    
    # Replace spaces between words that are likely part of an identifier with underscores
    def repl(m):
        return m.group(1) + '_' + m.group(2)

    # Apply the replacement repeatedly until no further replacements are made
    while True:
        s_new = pattern_identifier.sub(repl, s)
        if s_new == s:
            break
        s = s_new

    return s

def remove_unnecessary_characters(s):
    # First, remove specified unnecessary characters, keeping the dash to handle it separately
    s = re.sub(r'[\'`*<>~%$‘’"/]', '', s)
    # Replace "-" with "_", but avoid replacing it within " :- "
    # This uses a negative lookbehind and lookahead to ensure we don't replace dashes in " :- "
    s = re.sub(r'(?<!:)-', '_', s)  # Replace "-" with "_" if "-" is not immediately preceded by ":"
    s = re.sub(r'-(?! )', '_', s)  # Replace "-" with "_" if "-" is not immediately followed by a space, handling the case after ":-"
    
    return s

def remove_all_but_final_period(line):
    # Temporarily replace ":-" with a placeholder
    placeholder = "TEMP_COLON_HYPHEN"
    line = line.replace(':-', placeholder)

    # Remove all periods except the final one if it exists
    parts = line.rsplit('.', 1)
    if len(parts) == 2:
        line = parts[0].replace('.', '') + '.' + parts[1]
    else:
        line = parts[0].replace('.', '')

    # Restore ":-"
    line = line.replace(placeholder, ':-')
    return line

def filter_asp_code(asp_code):
    filtered_lines = []
    seen_lines = set()
    allowed_prefixes_pattern = re.compile(r'\b(neg|pos|relation)\(')

    for line in asp_code.split('\n'):
        # Add the processed line to the set of seen lines
        seen_lines.add(line)
        # Remove Non ASP code parts and special character
        if re.match(r'^\*\*|```asp', line):
            continue
        # Remove lines not in ASP format
        if not re.match(r'^[a-zA-Z0-9_].*', line):
            continue
        # Replace spaces with underscores in identifiers
        line = replace_spaces_with_underscores_in_identifiers(line)
        # remove lines with the word section in it
        if "section" in line.lower():
            continue
        if not "." in line.lower():
            continue
        if not allowed_prefixes_pattern.search(line):  # Skip lines not matching the allowed patterns
            continue
        # # Delete incomplete sentences
        # if not re.search(r'\)$', line):
        #     continue
        # Add missing full stops
        if not re.search(r'\.$', line) and re.search(r'\)$', line):
            line += '.'
        # Remove dots between words
        line = re.sub(r'(?<=\w)\.(?=\w)', '', line)
        # Convert all numeric digits to their corresponding words
        line = convert_number_to_words(line)
        # remove special characters
        line = remove_unnecessary_characters(line)
        # Lowercase conversion while keeping ASP keywords or variables intact
        line = re.sub(r'(?<![A-Z])([A-Z]+)(?![A-Z])', lambda x: x.group(1).lower(), line)
        # Ensure only the final period is kept
        line = remove_all_but_final_period(line)  
        filtered_lines.append(line)
    return '\n'.join(filtered_lines)
        

def apply_filter_to_asp_code(asp_code_file, output_file):
    with open(asp_code_file, 'r') as file:
        asp_code = file.read()

    filtered_code = filter_asp_code(asp_code)

    with open(output_file, 'w') as file:
        file.write(filtered_code)
    
    print(f"Filtered ASP code has been written to {output_file}")

In [6]:
# def num_to_words(match):
#     num_word_dict = {'0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', 
#                      '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine', '10': 'ten', 
#                      '15': 'fifteen'}
#     return num_word_dict.get(match.group(), match.group())

# def filter(output_file):
#     with open(output_file, 'r') as output:
#         lines = output.readlines()
#     processed_lines = []

#     for line in lines:
#         line = line.strip()

#         # Remove lines that are definitively incomplete
#         if not line or line.startswith(("section", ".", "*", "`", "%")):
#             continue

#         # Remove special characters
#         line = re.sub(r'[\$\é]', '', line)

#         # Ensure completeness for lines ending with specific structures
#         # if not (line.endswith(').') or line.endswith(')') or (line.endswith('.') and (' :- ' in line or line.startswith(('neg', 'pos', 'relation'))))):
#         #     continue

#         # Remove special characters, convert numbers to words, lowercase letters, and remove in-word periods
#         line = re.sub(r'[^\w\s.)]', '', line)  # Adjusted to keep periods and parentheses
#         line = re.sub(r'\b\d+\b', lambda x: num_to_words(x), line)
#         line = line.lower()
#         line = re.sub(r'(?<=\w)\.(?=\w)', '', line)

#         processed_lines.append(line)

#     with open(output_file, 'w') as output:
#         output.write('\n'.join(processed_lines))

In [7]:
def clingo_compiling_check(file):
    # Clingo program
    program = clingo.Control()
    # Reading ASP code
    with open(file, 'r') as f:
        asp_code = f.read()
    # Load ASP code
    program.add('base', [], asp_code)
    # Compiling and solving
    try:
        program.ground([("base", [])])
        program.solve()
        return True
    except Exception as e:
        # Returns False if the ASP code has a wrong syntax and is not compiling
        return False

In [8]:
def solve_asp(file_path):
    # Read the ASP code from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        asp_code = file.read()

    # Create a new clingo Control object
    solver = clingo.Control()
    
    # Add the ASP code to the solver
    solver.add("base", [], asp_code)
    
    # Ground the program
    solver.ground([("base", [])])
    
    # Define the on_model function to print models
    def on_model(model):
        print(f"Solution: {model}")
    
    # Solve the ASP program, calling on_model for each solution
    result = solver.solve(on_model=on_model)
    
    # Check if a solution exists
    if result.satisfiable:
        print("A solution exists.")
    else:
        print("No solution exists.")


In [14]:
# copy_file_content("PaLM output.txt", "filtered asp code.txt")

In [27]:
apply_filter_to_asp_code("PaLM output.txt", "filtered asp code.txt")

Filtered ASP code has been written to filtered asp code.txt


In [11]:
# Apply the filter function on the new file
# filter("filtered asp code.txt")

TypeError: filter expected 2 arguments, got 1

In [26]:
clingo_compiling_check("filtered asp code.txt")

<block>:34927:43-44: error: syntax error, unexpected ","

<block>:74630:135-136: error: syntax error, unexpected [

<block>:74631:131-132: error: syntax error, unexpected [

<block>:74632:133-134: error: syntax error, unexpected [

<block>:74633:131-132: error: syntax error, unexpected [

<block>:74634:125-126: error: syntax error, unexpected [



RuntimeError: parsing failed

In [17]:
solve_asp("filtered asp code.txt")

Solution: pos(pc,thousand_and_eighty_p) pos(pc,sixty_fps) pos(pc,choice_of_hardware_parts) pos(pc,not_made_by_single_company) pos(pc,can_stream_to_nvidia_shield) pos(pc,can_be_hooked_up_to_tv) pos(pc,can_be_used_with_a_controller) pos(pc,can_open_big_picture) pos(pc,can_browse_the_web) pos(pc,can_open_all_of_your_steam_games) pos(pc,no_disc) pos(video_games,story_telling) pos(video_games,control) pos(video_games,choices) pos(video_games,player_created_stories) pos(planetside_two,stories) pos(planetside_two,tension) pos(planetside_two,connection) pos(vegetarian,animals) pos(vegan,animals) pos(hunting,food) pos(farm,animals) pos(gun_control,stance) pos(gun_ownership,second_amendment) pos(hillary,first_woman_president) pos(rubio,first_latino_president) pos(rubio,young) pos(rubio,new) pos(rubio,fresh) pos(rubio,gop_version) pos(rubio,youth_driven_obamamania) pos(rubio,two_thousand_and_eight) pos(successful_people,troubled_beginnings) pos(troubled_beginnings,death_of_a_parent) pos(troubled_